# Introdução

Neste documentos iniciamos a extração das informações e do pdf dos documentos recuperados via busca na Base Minerva.

1. Acesso de Busca: https://minerva.ufrj.br/F/185DLENDIGLA9D814HHDJCPJ94ABCG929NQB2EPASCRXGQYC5P-09214?func=find-a-0
2. Especifiquei Tese ou Dissertação em um período recente
3. Na lateral tem a opção de selecionar cada artigo, ou em cima selecionar todos
4. Depois da seleção, na barra superior tem "Salvar/Email"
5. Salvamento no formato RIS

# Parte 0: Instalações

In [1]:
# %pip install PyPDF2
# %pip install rispy

# Parte 1: Parseador de Arquivo RIS

In [2]:
import os
import rispy
import re
import json

In [3]:
class RISParser:
    def parse_authors(self, tag_group):
        return tag_group['AU']

    def parse_keywords(self, tag_group):
        return tag_group['KW']

    def parse_urls(self, tag_group):
        return tag_group['UR']

    def parse_pdf_url(self, tag_group):
        urls = tag_group['UR']
        for url in urls:
            if url.endswith('.pdf'):
                return url

        return None

    def parse_id(self, tag_group):
        url = self.parse_pdf_url(tag_group)
        
        if url is None:
            return None
        else:
            return url.split('.')[-2].split('/')[-1]
    
    def parse_title(self, tag_group):
        title = tag_group['TI'][0]
        if title.endswith(' /'):
            title = title[:-2]
        return title

    def parse_bibliography_pages(self, tag_group):
        notes = tag_group['N1']
        
        for note in notes:
            if note.startswith("Bibliografia:"):
                search = re.search(r"([0-9]+)-([0-9]+)", note)
                pages = (int(search.group(1)), int(search.group(2)))
                return pages

    def parse_abstract(self, tag_group, language='PT'):
        abstracts = tag_group['AB']

        parsed_abstracts = {}
        current_language = None
        for line in abstracts:
            search = re.search(r"^\[([^\]]+)\]", line)
            
            if search is None:
                parsed_abstracts[current_language] += line
            else:
                current_language = search.group(1)
                size = len(current_language) + 3
                parsed_abstracts[current_language] = line[size:]

        return parsed_abstracts[language]
    
    def group_by_tag(self, text):
        text = re.sub(r"\n[\t ]+", " ", text)
        lines = text.split("\n")

        tags = {}
        for line in lines:
            tag = line[:2]
            text = line[6:]
            
            if tag in tags:
                tags[tag].extend([text])
            else:
                tags[tag] = [text]

        return tags
    
    def parse(self, text):
        tag_group = self.group_by_tag(text)

        response = {
            "title": self.parse_title(tag_group),
            "authors": self.parse_authors(tag_group),
            "abstract": self.parse_abstract(tag_group),
            "bibliography_pages": self.parse_bibliography_pages(tag_group),
            "keywords": self.parse_keywords(tag_group),
            "urls": self.parse_urls(tag_group),
            "pdf_url": self.parse_pdf_url(tag_group),
            "id": self.parse_id(tag_group)
        }

        return response

In [4]:
ris_parser = RISParser()

# Parte 2: Filenames em RIS da Base Minerva

In [5]:
folder_path = "./metadata_minerva"

In [6]:
filename_list = []
for filename in os.listdir(folder_path):
    if filename.endswith(".end"):
        filename_list.append(os.path.join(folder_path, filename))

filename_list = sorted(filename_list)

In [7]:
filename_list

['./metadata_minerva/minerva_dissertacao_mestrado.end',
 './metadata_minerva/minerva_dissertacao_mestrado_2020_2023.end',
 './metadata_minerva/minerva_tese_doutorado.end',
 './metadata_minerva/minerva_tese_doutorado_2020_2023.end']

# Parte 3: Parse and Save dos Metadados Individuais

In [8]:
def parse_and_save(filepath):
    with open(filepath, "r") as file:
        file_text = file.read()
        files = file_text.split("\n\n")[1:]

        print("Size:", len(files))
        
        for f in files:
            try:
                parsed_metadata = ris_parser.parse(f)
                _id = parsed_metadata["id"]
                
                with open(f"metadata_documents/documento_{_id}.json", "w") as file:
                    json.dump(parsed_metadata, file, ensure_ascii=False).encode('utf8')

            except:
                print("Error")

In [ ]:
for filename in filename_list:
    print(">", filename)
    parse_and_save(filename)